In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .not(has("namespace","kube-system"))
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(1)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(elementMap())


kh.containers()
 .has("runID", graph.variables().get('runID_yourid').get().trim())
  .has("app",graph.variables().get('containerEscape_vulnApp_yourid').get().trim())
   .repeat( outE().inV().simplePath() // Building the path from one vertex to another
           ).until( has(label, "Node") // Stop when meeting a critical asset
    .or().loops().is(1) // Stop after X iteration
  ).has(label, "Node") // Keep only path ending with a critical asset
  .path().by(elementMap())
  .limit(1000)

In [2]:
from stix2 import CustomObject, properties

@CustomObject('x-kubernetes-pod', [
    ('name', properties.StringProperty(required=True)),
    ('namespace', properties.StringProperty(required=True)),
])
class KubernetesPod(object):
    pass

@CustomObject('x-kubernetes-node', [
    ('name', properties.StringProperty(required=True)),
])
class KubernetesNode(object):
    pass

@CustomObject('x-user-account', [
    ('user_id', properties.StringProperty(required=True)),
    ('permissions', properties.StringProperty(required=True)),
])
class UserAccount(object):
    pass


@CustomObject('x-tetragon-log', [
    ('event', properties.StringProperty(required=True)),
    ('details', properties.StringProperty(required=True)),
])
class TetragonLog(object):
    pass

import stix2
import uuid

def create_container_escape_indicator(path, tetragon_log, escape_type="GENERIC"):
    """
    Creates a STIX 2.1 Indicator object for container escape detection.

    Args:
        path (list): A list of dictionaries representing the path from the container to the node.
        tetragon_log (dict): A dictionary representing the Tetragon log evidence.
        escape_type (str, optional): The type of container escape. Defaults to "GENERIC".

    Returns:
        stix2.Indicator: The STIX 2.1 Indicator object.
    """

    container_data = path[0]
    node_data = path[1]

    # Extract relevant data from the path
    pod_name = container_data['pod']
    namespace = container_data['namespace']
    node_name = node_data['name']

    # Create STIX objects
    pod = KubernetesPod(
        id=f"x-kubernetes-pod--{uuid.uuid4()}",
        name=pod_name,
        namespace=namespace
    )

    node = KubernetesNode(
        id=f"x-kubernetes-node--{uuid.uuid4()}",
        name=node_name
    )

    user_account = UserAccount(
        id=f"x-user-account--{uuid.uuid4()}",
        user_id="some-user-id",
        permissions="SYS_PTRACE"
    )

    tetragon_log_obj = TetragonLog(
        id=f"x-tetragon-log--{uuid.uuid4()}",
        event=tetragon_log['event'],
        details=tetragon_log['details']
    )

    # Construct the STIX pattern based on escape type
    if escape_type == "SYS_PTRACE":
        pattern = f"[x-kubernetes-pod:name = '{pod_name}' AND x-kubernetes-pod:namespace = '{namespace}' AND x-user-account:permissions = 'SYS_PTRACE' AND x-kubernetes-node:name = '{node_name}' AND x-tetragon-log:event = '{tetragon_log['event']}' AND x-tetragon-log:details = '{tetragon_log['details']}']"
    elif escape_type == "NSENTER":
        pattern = f"[x-kubernetes-pod:name = '{pod_name}' AND x-kubernetes-pod:namespace = '{namespace}' AND x-kubernetes-pod:privileged = true AND x-kubernetes-pod:host_pid = true AND x-kubernetes-node:name = '{node_name}' AND x-tetragon-log:event = '{tetragon_log['event']}' AND x-tetragon-log:details = '{tetragon_log['details']}']"
    else:  # Generic escape pattern
        pattern = f"[x-kubernetes-pod:name = '{pod_name}' AND x-kubernetes-pod:namespace = '{namespace}' AND x-tetragon-log:event = '{tetragon_log['event']}' AND x-tetragon-log:details = '{tetragon_log['details']}']"

    # Create the STIX Indicator object
    indicator = stix2.Indicator(
        name=f"Container Escape ({escape_type})",
        description=f"Detects container escape from pod {pod_name} in namespace {namespace} using {escape_type}",
        pattern=pattern,
        pattern_type="stix",
        valid_from="2023-10-27T12:34:56Z"  # Update with the appropriate validity period
    )

    return indicator

# Example usage:
path = [
    {
        'runAsUser': 0, 'capabilities': '[SYS_PTRACE, SYS_ADMIN]', 'ports': '[]', 'class': 'Container',
        'cluster': 'gke_adls-152q5v4ovh2urxs17pnkwnlwl_europe-west1_k8s-caas-0009-beta', 'runID': '01ja8e863mqb3echwy4w4cer83',
        'storeID': '670e98db9ceaeac8428ae6d2', 'app': 'pl-monitoring', 'team': '', 'service': '', 'name': 'pem',
        'namespace': 'pl', 'isNamespaced': True, 'compromised': 0, 'node': 'gke-k8s-caas-0009-be-default-node-po-f38ee065-xqvf',
        'image': 'gcr.io/pixie-oss/pixie-prod/vizier-pem_image:0.14.11', 'pod': 'vizier-pem-h6lk9', 'hostNetwork': True,
        'hostPid': True, 'hostIpc': False, 'privesc': False, 'privileged': True
    },
    {
        'class': 'Node', 'cluster': 'gke_adls-152q5v4ovh2urxs17pnkwnlwl_europe-west1_k8s-caas-0009-beta', 'runID': '01ja8e863mqb3echwy4w4cer83',
        'storeID': '670e98d99ceaeac8428ae491', 'app': '', 'team': '', 'service': '', 'name': 'gke-k8s-caas-0009-be-default-node-poo-f38ee065-xqvf',
        'namespace': '', 'critical': False, 'isNamespaced': False, 'compromised': 0
    }
]

tetragon_log = {
    'event': 'namespace_change',
    'details': 'Detected change in Linux namespaces'
}

sys_ptrace_indicator = create_container_escape_indicator(path, tetragon_log, escape_type="SYS_PTRACE")
nsenter_indicator = create_container_escape_indicator(path, tetragon_log, escape_type="NSENTER")
generic_indicator = create_container_escape_indicator(path, tetragon_log)

print(sys_ptrace_indicator)
print(nsenter_indicator)
print(generic_indicator)

{"type": "indicator", "spec_version": "2.1", "id": "indicator--5a649969-1c6c-4ae1-88d3-573f2d373b0c", "created": "2024-10-17T21:38:14.121637Z", "modified": "2024-10-17T21:38:14.121637Z", "name": "Container Escape (SYS_PTRACE)", "description": "Detects container escape from pod vizier-pem-h6lk9 in namespace pl using SYS_PTRACE", "pattern": "[x-kubernetes-pod:name = 'vizier-pem-h6lk9' AND x-kubernetes-pod:namespace = 'pl' AND x-user-account:permissions = 'SYS_PTRACE' AND x-kubernetes-node:name = 'gke-k8s-caas-0009-be-default-node-poo-f38ee065-xqvf' AND x-tetragon-log:event = 'namespace_change' AND x-tetragon-log:details = 'Detected change in Linux namespaces']", "pattern_type": "stix", "pattern_version": "2.1", "valid_from": "2023-10-27T12:34:56Z"}
{"type": "indicator", "spec_version": "2.1", "id": "indicator--f16c67d8-df6b-428e-83d2-ccaec01d2e43", "created": "2024-10-17T21:38:14.151967Z", "modified": "2024-10-17T21:38:14.151967Z", "name": "Container Escape (NSENTER)", "description": "De

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .has("app",graph.variables().get('containerEscape_vulnApp_yourid').get().trim())
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(1)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(elementMap())
    .limit(1000)

In [1]:
pods = [
{'runAsUser': [10000], 'capabilities': ['[]'], 'ports': ['[8080]'], 'class': ['Container'], 'cluster': ['gke_adls-152q5v4ovh2urxs17pnkwnlwl_europe-west1_k8s-caas-0009-beta'], 'runID': ['01ja8e863mqb3echwy4w4cer83'], 'storeID': ['670e98da9ceaeac8428ae4df'], 'app': ['harbor'], 'team': [''], 'service': [''], 'name': ['jobservice'], 'namespace': ['harbor'], 'isNamespaced': [True], 'compromised': [0], 'node': ['gke-k8s-caas-0009-be-default-node-poo-f38ee065-p3pi'], 'image': ['goharbor/harbor-jobservice:v2.11.1'], 'pod': ['harbor-jobservice-6c667d97db-qxgzk'], 'hostNetwork': [False], 'hostPid': [False], 'hostIpc': [False], 'privesc': [False], 'privileged': [False]},
{'runAsUser': [0], 'command': ['[/gke-metadata-server, --logtostderr, --token-exchange-endpoint=https://securetoken.googleapis.com/v1/identitybindingtoken, --workload-pool=adls-152q5v4ovh2urxs17pnkwnlwl.svc.id.goog, --alts-service-suffixes-using-node-identity=storage.googleapis.com,bigtable.googleapis.com,bigtable2.googleapis.com,bigtablerls.googleapis.com,spanner.googleapis.com,spanner2.googleapis.com,spanner-rls.googleapis.com,grpclb.directpath.google.internal,grpclb-dualstack.directpath.google.internal,staging-wrenchworks.sandbox.googleapis.com,preprod-spanner.sandbox.googleapis.com,wrenchworks-loadtest.googleapis.com,wrenchworks-nonprod.googleapis.com, --identity-provider=https://container.googleapis.com/v1/projects/adls-152q5v4ovh2urxs17pnkwnlwl/locations/europe-west1/clusters/k8s-caas-0009-beta, --passthrough-ksa-list=anthos-identity-service:gke-oidc-envoy-sa,anthos-identity-service:gke-oidc-service-sa,gke-managed-dpv2-observability:hubble-relay,kube-system:antrea-controller,kube-system:container-watcher-pod-reader,kube-system:coredns,kube-system:egress-nat-controller,kube-system:event-exporter-sa,kube-system:fluentd-gcp-scaler,kube-system:gke-metrics-agent,kube-system:gke-spiffe-node-agent,kube-system:heapster,kube-system:konnectivity-agent,kube-system:kube-dns,kube-system:maintenance-handler,kube-system:metadata-agent,kube-system:network-metering-agent,kube-system:node-local-dns,kube-system:pkgextract-service,kube-system:pkgextract-cleanup-service,kube-system:securityprofile-controller,istio-system:istio-ingressgateway-service-account,istio-system:cluster-local-gateway-service-account,csm:csm-sync-agent,knative-serving:controller,kube-system:pdcsi-node-sa,kube-system:gcsfusecsi-node-sa,gmp-system:collector,gke-gmp-system:collector,gke-managed-cim:kube-state-metrics, --attributes=cluster-name=k8s-caas-0009-beta,cluster-uid=28d64db24b1d4dfeb55612f28174511c49836b27d68c4bfc923d7423ee4dca9d,cluster-location=europe-west1, --cluster-uid=28d64db24b1d4dfeb55612f28174511c49836b27d68c4bfc923d7423ee4dca9d, --sts-endpoint=https://sts.googleapis.com, --token-exchange-mode=sts, --cloud-monitoring-endpoint=monitoring.googleapis.com:443, --iam-cred-service-endpoint=https://iamcredentials.googleapis.com, --cluster-project-number=601855464707, --cluster-location=europe-west1, --cluster-name=k8s-caas-0009-beta, --component-version=0.4.302, --ksa-cache-mode=watchchecker, --kcp-allow-watch-checker=true, --csi-socket=/csi/csi.sock, --volumes-db=/var/run/gkemds.gke.io/csi/volumes.boltdb]'], 'capabilities': ['[]'], 'ports': ['[987, 988, 989]'], 'class': ['Container'], 'cluster': ['gke_adls-152q5v4ovh2urxs17pnkwnlwl_europe-west1_k8s-caas-0009-beta'], 'runID': ['01ja8e863mqb3echwy4w4cer83'], 'storeID': ['670e98da9ceaeac8428ae571'], 'app': [''], 'team': [''], 'service': [''], 'name': ['gke-metadata-server'], 'namespace': ['kube-system'], 'isNamespaced': [True], 'compromised': [0], 'node': ['gke-k8s-caas-0009-be-default-node-poo-f38ee065-p3pi'], 'image': ['gke.gcr.io/gke-metadata-server:gke_metadata_server_20240702.00_p0@sha256:aea9cc887c91b9a05e5bb4bb604180772594a01f0828bbfacf30c77562ac7205'], 'pod': ['gke-metadata-server-pz4zp'], 'hostNetwork': [True], 'hostPid': [False], 'hostIpc': [False], 'privesc': [False], 'privileged': [True]},
]